In [320]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit as qk
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram
import random
from collections import Counter

from generator import generateCliffordCircuit
# from execution.transpiler import transpileListToQiskitCircuit

In [321]:
def stringToQiskitSingleGate(gateString, qiskitCir, whichQubit):
    if gateString == 'I':
        qiskitCir.id(whichQubit)
    elif gateString == 'X':
        qiskitCir.x(whichQubit)
    elif gateString == 'Y':
        qiskitCir.y(whichQubit)
    elif gateString == 'Z':
        qiskitCir.z(whichQubit)
    elif gateString == 'H':
        qiskitCir.h(whichQubit)
    elif gateString == 'S':
        qiskitCir.s(whichQubit)
        
def transpileListToQiskitCircuit(cir):
    depth = len(cir)
    width = len(cir[0])
    qiskitCir = qk.QuantumCircuit(width)
    for d in range(width):
        if d % 2 == 0:
            for w in range(width):
                singleGate = cir[d][w]
                stringToQiskitSingleGate(singleGate, qiskitCir, w)
            if d != width - 1:
                qiskitCir.barrier()
        else:
            c = cir[d].index('CNOT_C')
            t = cir[d].index('CNOT_T')
            qiskitCir.cx(c, t)
            if d != width - 1:
                qiskitCir.barrier()
    return qiskitCir

In [322]:
width = 5 # num_qubits
depth = 5 # number of layers in the circuit (easy + hard)
singleGateSet = ['X', 'H', 'Z', 'I', 'S']
doubleGateSet = ['CNOT_C', 'CNOT_T']
twirlingGateSet = ['X','Y' 'Z', 'I']

circuit = generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet)
qiskitCircuit = transpileListToQiskitCircuit(circuit)
print(circuit)
if depth < 10:
    print(qiskitCircuit)

[['I', 'Z', 'I', 'Z', 'Z'], ['I', 'I', 'I', 'CNOT_T', 'CNOT_C'], ['I', 'Z', 'H', 'H', 'X'], ['CNOT_T', 'I', 'CNOT_C', 'I', 'I'], ['H', 'I', 'X', 'H', 'H']]
     ┌───┐ ░       ░ ┌───┐ ░ ┌───┐ ░ ┌───┐
q_0: ┤ I ├─░───────░─┤ I ├─░─┤ X ├─░─┤ H ├
     ├───┤ ░       ░ ├───┤ ░ └─┬─┘ ░ ├───┤
q_1: ┤ Z ├─░───────░─┤ Z ├─░───┼───░─┤ I ├
     ├───┤ ░       ░ ├───┤ ░   │   ░ ├───┤
q_2: ┤ I ├─░───────░─┤ H ├─░───■───░─┤ X ├
     ├───┤ ░ ┌───┐ ░ ├───┤ ░       ░ ├───┤
q_3: ┤ Z ├─░─┤ X ├─░─┤ H ├─░───────░─┤ H ├
     ├───┤ ░ └─┬─┘ ░ ├───┤ ░       ░ ├───┤
q_4: ┤ Z ├─░───■───░─┤ X ├─░───────░─┤ H ├
     └───┘ ░       ░ └───┘ ░       ░ └───┘


In [323]:
def Clifford_Permute(cliff,pauli,qargs = None):
    '''This function permutes the pauli operators according to the clifford group, so it computes 
    P' = C P C^T, where C is a clifford operator and P is a pauli operator.
    Inputs: cliff - A Qiskit Circuit, Clifford, or Gate object.
            pauli - A Qiskit Pauli object.
    Outputs: new_pauli - A Qiskit Pauli object.'''

    if type(pauli) == qk.quantum_info.Pauli:
        input_pauli_string = ''.join(char for char in pauli.to_label() if char.isalpha())
        input_pauli_string = input_pauli_string[::-1]
        p = qk.quantum_info.Pauli(input_pauli_string)
    elif type(pauli) == str:
        input_pauli_string = ''.join(char for char in pauli if char.isalpha())
        input_pauli_string = input_pauli_string[::-1]
        p = qk.quantum_info.Pauli(input_pauli_string)

    return p.evolve(cliff,qargs,frame="s")


In [324]:
Clifford_Permute(qiskitCircuit, qk.quantum_info.Pauli('X'*width))

Pauli('-ZIZXZ')

In [325]:
def split_circuit_by_barrier(circuit):
    qasm = circuit.qasm()
    prelude = []
    circuits = [[]]
    for line in qasm.splitlines():
        if any([line.startswith(t) for t in ['OPENQASM', 'include', 'qreg', 'creg']]):
            prelude.append(line)
        elif line.startswith('barrier'):
            circuits.append([])
        else:
            circuits[-1].append(line)
    circuits_with_prelude = [prelude+circuit for circuit in circuits]
    for circuit_with_prelude in circuits_with_prelude:
        yield qk.QuantumCircuit.from_qasm_str('\n'.join(circuit_with_prelude))

# for subcircuit in (split_circuit_by_barrier(qiskitCircuit)):
#     print(subcircuit)

In [326]:
def G_twirling(circuit):
    '''This function performs G-twirling on a circuit. It randomly chooses a Pauli from {X,Y,Z,I} and prepends it before a layer, then appends P'=CPC^T after the layer.}
    Inputs: circuit - A Qiskit Circuit object.
    Outputs: new_circuit - A Qiskit Circuit object.'''
    num_qubits = circuit.num_qubits
    new_circuit = qk.QuantumCircuit(num_qubits)
    for index, subcircuit in enumerate(split_circuit_by_barrier(circuit)):
        # if len(subcircuit.data) == 1:
        #     operated_qubits = [subcircuit.find_bit(q).index for q in subcircuit.data[0].qubits]
        # else:
        #     operated_qubits = list(range(num_qubits))
        pauli_str = ''.join(random.choice(["X","Y","Z","I"]) for _ in range(num_qubits))
        pauli = qk.quantum_info.Pauli(pauli_str[::-1])
        new_circuit = new_circuit.compose(pauli,qubits=range(num_qubits))
        new_circuit = new_circuit.compose(subcircuit,qubits=range(num_qubits))
        new_circuit = new_circuit.compose(Clifford_Permute(subcircuit,pauli).to_instruction(),qubits = range(num_qubits))
        if index != len(list(split_circuit_by_barrier(circuit)))-1:
            new_circuit.barrier()
    return new_circuit


In [327]:
G_twirling(qiskitCircuit).draw()

┌───────────────┐┌───┐┌─────────┐ ░ ┌───────────────┐     »
q_0: ┤0              ├┤ I ├┤0        ├─░─┤0              ├─────»
     │               │├───┤│         │ ░ │               │     »
q_1: ┤1              ├┤ Z ├┤1        ├─░─┤1              ├─────»
     │               │├───┤│         │ ░ │               │     »
q_2: ┤2 Pauli(IXXZZ) ├┤ I ├┤2 -ZZXXI ├─░─┤2 Pauli(IZZXY) ├─────»
     │               │├───┤│         │ ░ │               │┌───┐»
q_3: ┤3              ├┤ Z ├┤3        ├─░─┤3              ├┤ X ├»
     │               │├───┤│         │ ░ │               │└─┬─┘»
q_4: ┤4              ├┤ Z ├┤4        ├─░─┤4              ├──■──»
     └───────────────┘└───┘└─────────┘ ░ └───────────────┘     »
«     ┌───────────────┐ ░ ┌───────────────┐┌───┐┌─────────┐ ░ ┌───────────────┐»
«q_0: ┤0              ├─░─┤0              ├┤ I ├┤0        ├─░─┤0              ├»
«     │               │ ░ │               │├───┤│         │ ░ │               │»
«q_1: ┤1              ├─░─┤1              ├┤ Z ├┤1        ├─░─┤1              ├»
«     │               │ ░ │               │├───┤│         │ ░ │               │»
«q_2: ┤2 Pauli(YIZZI) ├─░─┤2 Pauli(ZXXXX) ├┤ H ├┤2 -XZZXZ ├─░─┤2 Pauli(ZZXYZ) ├»
«     │               │ ░ │               │├───┤│         │ ░ │               │»
«q_3: ┤3              ├─░─┤3              ├┤ H ├┤3        ├─░─┤3              ├»
«     │               │ ░ │               │├───┤│         │ ░ │               │»
«q_4: ┤4              ├─░─┤4              ├┤ X ├┤4        ├─░─┤4              ├»
«     └───────────────┘ ░ └───────────────┘└───┘└─────────┘ ░ └───────────────┘»
«     ┌───┐┌─────────┐ ░ ┌───────────────┐┌───┐┌───────────────┐
«q_0: ┤ X ├┤0        ├─░─┤0              ├┤ H ├┤0              ├
«     └─┬─┘│         │ ░ │               │├───┤│               │
«q_1: ──┼──┤1        ├─░─┤1              ├┤ I ├┤1              ├
«       │  │         │ ░ │               │├───┤│               │
«q_2: ──■──┤2 -ZYYZY ├─░─┤2 Pauli(YZYXI) ├┤ X ├┤2 Pauli(IZYZY) ├
«          │         │ ░ │               │├───┤│               │
«q_3: ─────┤3        ├─░─┤3              ├┤ H ├┤3              ├
«          │         │ ░ │               │├───┤│               │
«q_4: ─────┤4        ├─░─┤4              ├┤ H ├┤4              ├
«          └─────────┘ ░ └───────────────┘└───┘└───────────────┘

In [328]:
circuit_ensemble = [G_twirling(qiskitCircuit) for _ in range(10)]
# for circ in circuit_ensemble:
#     print(circ)

In [329]:
def random_choices_with_counts(items, n):
    """
    Randomly choose with replacement n items from a list and return a dictionary
    with the counts of each chosen item.
    
    Parameters:
        items (list): List of items to choose from.
        n (int): Number of items to choose.
    
    Returns:
        dict: Dictionary with the counts of each chosen item.
    """
    chosen_items = random.choices(items, k=n)
    counts = Counter(chosen_items)
    return dict(counts)

In [330]:
def find_circuit_from_name(circuit_list,target_name):
    # Find the circuit with the given name
    found_circuit = None
    for circuit in circuit_list:
        if circuit.name == target_name:
            found_circuit = circuit
            break

    # Check if the circuit was found
    if found_circuit is not None:
        # print(f"Circuit with name '{target_name}' found:")
        # print(found_circuit)
        return found_circuit
    else:
        print(f"No circuit with name '{target_name}' found.")
        return found_circuit

In [331]:
me = qk.QuantumCircuit(5)
me.pauli(pauli_string = "XX",qubits = [1,3])
print(me)

                   
q_0: ──────────────
     ┌────────────┐
q_1: ┤0           ├
     │            │
q_2: ┤  Pauli(XX) ├
     │            │
q_3: ┤1           ├
     └────────────┘
q_4: ──────────────
                   


In [332]:
num_experiments = 1000
freq_dict = random_choices_with_counts([circ.name for circ in circuit_ensemble],num_experiments)
find_circuit_from_name(circuit_ensemble,list(freq_dict.keys())[0])

In [333]:
def prep_circuit(input_pauli):
    '''This function prepares the input state for eigenvalue sampling. It takes in a Pauli operator to be input into the circuit and outputs a circuit that prepares the state |psi+> or |psi-> from the input |0>^n, 
    where |psi+> and |psi-> are the eigenstates of the input Pauli operator.
    Inputs: input_pauli - A Qiskit Pauli object or a string representing a Pauli operator.
    Outputs: prep_circ - A Qiskit Circuit object which will prepare the state |psi+> or |psi-> from |0>^n, where |psi+> and |psi-> are the eigenstates of the input Pauli operator.
    '''
    # mapping for preparation circuit. The idea of the prep circuit is that we append it before the circuit to eigenvalue samp
    # i.e. input the state where all qubits are zero 1000...0> into the prep circuit and then input it's ouput into the circu
    # |0> -> ...
    # H -> |+>
    # XH -> |->
    # HS -> |+i>
    # XHS -> |-i>
    # I -> |0> (no change)
    # X -> |1>
    if type(input_pauli) == str:
        input_pauli_string = ''.join(char for char in input_pauli if char.isalpha()) # removes the phase from the Pauli string if present
    elif type(input_pauli) == qk.quantum_info.Pauli:
        input_pauli_string = ''.join(char for char in input_pauli.to_label() if char.isalpha()) # converts the Pauli to a string w/o phase
        input_pauli_string = input_pauli_string[::-1] # reverses the string so that the first qubit is the first character in the string
    
    nontriv_indices = [index for index, char in enumerate(input_pauli_string) if char != 'I'] # finds the indices of the nontrivial Pauli gates

    prep_circ = qk.QuantumCircuit(len(input_pauli_string))
    p_eigenstate = [random.choice(range(2)) for i in nontriv_indices] # randomly chooses |psi+> (0) or |psi-> (1)
    
    for index, qubit in enumerate(nontriv_indices):
        pauli_gate = input_pauli_string[qubit]
        if pauli_gate == "X": # if the input Pauli is X
            if p_eigenstate[index] == 0: # if the randomly chosen eigenvector is |+>
                prep_circ.h(qubit)
            elif p_eigenstate[index] == 1: # if the randomly chosen eigenvector is |->
                prep_circ.x(qubit)
                prep_circ.h(qubit)
        elif pauli_gate == "Y": # if the input Pauli is Y
            if p_eigenstate[index] == 0: # if the randomly chosen eigenvector is l+i>
                prep_circ.h(qubit)
                prep_circ.s(qubit)
            elif p_eigenstate[index] == 1: # if the randomly chosen eigenvector is |-i>
                prep_circ.x(qubit)
                prep_circ.h(qubit)
                prep_circ.s(qubit)
        elif pauli_gate == "Z": # if the input Pauli is Z
            # we don't need to append anything if the chosen eigenstate is 10>, since the input will already be in that state
            if p_eigenstate[index] == 1: # if the randomly chosen eigenvector is
                prep_circ.x(qubit)

    p_in_eigenstate = sum(p_eigenstate) % 2
    return prep_circ, p_in_eigenstate

In [334]:
def measure_circuit(final_pauli):
    if type(final_pauli) == str:
        final_pauli_string = ''.join(char for char in final_pauli if char.isalpha()) # removes the phase from the Pauli string if present
    elif type(final_pauli) == qk.quantum_info.Pauli:
        final_pauli_string = ''.join(char for char in final_pauli.to_label() if char.isalpha()) # converts the Pauli to a string w/o phase
        final_pauli_string = final_pauli_string[::-1] # reverses the string so that the first qubit is the first character in the string

    nontriv_indices = [index for index, char in enumerate(final_pauli_string) if char != 'I'] # finds the indices of the nontrivial Pauli gates
    nontriv_gates = [final_pauli_string[index] for index in nontriv_indices]

    measurement_circ = qk.QuantumCircuit(len(final_pauli_string),len(nontriv_indices))
    # On the measurement circ, we need to dagger the transforms used to prepare the prep circ to measure in the bases given by P'
    for index, g in enumerate(nontriv_gates):
        # print(index)
        if g == "X":
            measurement_circ.h(nontriv_indices[index]) # rotate into X basis
            measurement_circ.measure(nontriv_indices[index],index) # then measure
        elif g == "Y":
            measurement_circ.sdg(nontriv_indices[index])
            measurement_circ.h(nontriv_indices[index]) # S**-1 H rotates into Y basis
            measurement_circ.measure(nontriv_indices[index],index) # then measure
        elif g == "Z":
            measurement_circ.measure(nontriv_indices[index],index) # measure

    return measurement_circ

In [335]:
st = "XYIZX"
b,c = prep_circuit(st)
print(b)
a = measure_circuit(st)
print(a)
print("Number of Classical Bits: ",a.num_clbits)

     ┌───┐┌───┐     
q_0: ┤ X ├┤ H ├─────
     ├───┤├───┤┌───┐
q_1: ┤ X ├┤ H ├┤ S ├
     └───┘└───┘└───┘
q_2: ───────────────
     ┌───┐          
q_3: ┤ X ├──────────
     ├───┤┌───┐     
q_4: ┤ X ├┤ H ├─────
     └───┘└───┘     
      ┌───┐      ┌─┐      
q_0: ─┤ H ├──────┤M├──────
     ┌┴───┴┐┌───┐└╥┘   ┌─┐
q_1: ┤ Sdg ├┤ H ├─╫────┤M├
     └─────┘└───┘ ║    └╥┘
q_2: ─────────────╫─────╫─
       ┌─┐        ║     ║ 
q_3: ──┤M├────────╫─────╫─
       └╥┘  ┌───┐ ║ ┌─┐ ║ 
q_4: ───╫───┤ H ├─╫─┤M├─╫─
        ║   └───┘ ║ └╥┘ ║ 
c: 4/═══╩═════════╩══╩══╩═
        2         0  3  1 
Number of Classical Bits:  4


In [336]:
def est_Lambda(input_pauli,circuit_ensemble,num_experiments):

    if type(input_pauli) == str:
        input_pauli_string = ''.join(char for char in input_pauli if char.isalpha()) # removes the phase from the Pauli string if present
    elif type(input_pauli) == qk.quantum_info.Pauli:
        input_pauli_string = ''.join(char for char in input_pauli.to_label() if char.isalpha()) # converts the Pauli to a string w/o phase
        input_pauli_string = input_pauli_string[::-1] # reverses the string so that the first qubit is the first character in the string

    freq_dict = random_choices_with_counts([circ.name for circ in circuit_ensemble],num_experiments)

    eigenresults_dict = {("+","+"):0,("+","-"):0,("-","+"):0,("-","-"):0}
    for circ_name in freq_dict:
        circ = find_circuit_from_name(circuit_ensemble,circ_name)
        num_shots = freq_dict[circ_name]
        start_in_plus = random.randint(num_shots*.1, num_shots)
        starter = [start_in_plus, num_shots - start_in_plus]
        
        p_in_eigenstate = 100
        for i in [0,1]:
            while p_in_eigenstate != i:
                prep_circ,p_in_eigenstate = prep_circuit(input_pauli_string)
        
            if p_in_eigenstate == 0:
                p_eigenstate = "+"
            elif p_in_eigenstate == 1:
                p_eigenstate = "-"
            
            prep_circ.barrier()

            measure_circ = measure_circuit(Clifford_Permute(circ,input_pauli_string))

            temp_circuit = prep_circ.compose(circ)
            temp_circuit.barrier()
            full_circuit = temp_circuit.compose(measure_circ)
            # print(full_circuit)
        
            # Simulate the circuit with the stabilizer simulator
            simulator = Aer.get_backend('aer_simulator', method='stabilizer')
            result = qk.execute(full_circuit, backend = simulator, shots = starter[i]).result()
            counts = result.get_counts()

            for r in counts:
                parity = sum(int(x) for x in r if x.isdigit()) % 2
                if parity == 0:
                    pp_eigenstate = "+"
                elif parity == 1:
                    pp_eigenstate= "-"

                if (p_eigenstate,pp_eigenstate) not in eigenresults_dict:
                    eigenresults_dict[(p_eigenstate,pp_eigenstate)] = counts[r]
                else:
                    eigenresults_dict[(p_eigenstate,pp_eigenstate)] += counts[r]

    # print(sum(eigenresults_dict.values()))
    # Lambda_est = ((eigenresults_dict[("+","+")] - eigenresults_dict[("+","-")])/(eigenresults_dict[("+","+")] + eigenresults_dict[("+","-")]) - (eigenresults_dict[("-","+")] - eigenresults_dict[("-","-")])/(eigenresults_dict[("-","+")] + eigenresults_dict[("-","-")]))
    Lambda_est = ((eigenresults_dict[("+","+")] - eigenresults_dict[("+","-")]) - (eigenresults_dict[("-","+")] - eigenresults_dict[("-","-")]))/sum(eigenresults_dict.values())

    return Lambda_est

In [337]:
out = est_Lambda(st,[qiskitCircuit],1000)
print(out)

1.0


In [338]:
Clifford_Permute(qiskitCircuit, st)

Pauli('YYIYZ')

In [339]:
count_dict = {0:0,1:0}
for i in range(1):
    full_circuit = qk.QuantumCircuit(qiskitCircuit.num_qubits)
    full_circuit.initialize(0)
    
    prep_circ,p_in_eigenstate = prep_circuit(st)
    prep_circ.barrier()
    count_dict[p_in_eigenstate] += 1
    measure_circ = measure_circuit(Clifford_Permute(qiskitCircuit,st))

    temp_circuit = prep_circ.compose(qiskitCircuit)
    temp_circuit.barrier()
    # print(temp_circuit)
    full_circuit = temp_circuit.compose(measure_circ)
    print(full_circuit)
    print("Number of Classical Bits: ", full_circuit.num_clbits)

    simulator = Aer.get_backend('aer_simulator', method='stabilizer')
    result = qk.execute(full_circuit, backend = simulator, shots = 100).result()
    counts = result.get_counts()
    
    print(p_in_eigenstate)
    print(counts)
    print(result)

     ┌───┐┌───┐      ░ ┌───┐ ░       ░ ┌───┐ ░ ┌───┐ ░ ┌───┐ ░ ┌─┐            »
q_0: ┤ X ├┤ H ├──────░─┤ I ├─░───────░─┤ I ├─░─┤ X ├─░─┤ H ├─░─┤M├────────────»
     ├───┤├───┤┌───┐ ░ ├───┤ ░       ░ ├───┤ ░ └─┬─┘ ░ ├───┤ ░ └╥┘┌─────┐┌───┐»
q_1: ┤ X ├┤ H ├┤ S ├─░─┤ Z ├─░───────░─┤ Z ├─░───┼───░─┤ I ├─░──╫─┤ Sdg ├┤ H ├»
     └───┘└───┘└───┘ ░ ├───┤ ░       ░ ├───┤ ░   │   ░ ├───┤ ░  ║ └─────┘└───┘»
q_2: ────────────────░─┤ I ├─░───────░─┤ H ├─░───■───░─┤ X ├─░──╫─────────────»
                     ░ ├───┤ ░ ┌───┐ ░ ├───┤ ░       ░ ├───┤ ░  ║ ┌─────┐┌───┐»
q_3: ────────────────░─┤ Z ├─░─┤ X ├─░─┤ H ├─░───────░─┤ H ├─░──╫─┤ Sdg ├┤ H ├»
     ┌───┐           ░ ├───┤ ░ └─┬─┘ ░ ├───┤ ░       ░ ├───┤ ░  ║ ├─────┤├───┤»
q_4: ┤ H ├───────────░─┤ Z ├─░───■───░─┤ X ├─░───────░─┤ H ├─░──╫─┤ Sdg ├┤ H ├»
     └───┘           ░ └───┘ ░       ░ └───┘ ░       ░ └───┘ ░  ║ └─────┘└───┘»
c: 4/═══════════════════════════════════════════════════════════╩═════════════»
                                        

In [340]:
print(full_circuit)
# prep_circ.barrier()
me = prep_circ.compose(qiskitCircuit)
print(me.compose(measure_circ))

     ┌───┐┌───┐      ░ ┌───┐ ░       ░ ┌───┐ ░ ┌───┐ ░ ┌───┐ ░ ┌─┐            »
q_0: ┤ X ├┤ H ├──────░─┤ I ├─░───────░─┤ I ├─░─┤ X ├─░─┤ H ├─░─┤M├────────────»
     ├───┤├───┤┌───┐ ░ ├───┤ ░       ░ ├───┤ ░ └─┬─┘ ░ ├───┤ ░ └╥┘┌─────┐┌───┐»
q_1: ┤ X ├┤ H ├┤ S ├─░─┤ Z ├─░───────░─┤ Z ├─░───┼───░─┤ I ├─░──╫─┤ Sdg ├┤ H ├»
     └───┘└───┘└───┘ ░ ├───┤ ░       ░ ├───┤ ░   │   ░ ├───┤ ░  ║ └─────┘└───┘»
q_2: ────────────────░─┤ I ├─░───────░─┤ H ├─░───■───░─┤ X ├─░──╫─────────────»
                     ░ ├───┤ ░ ┌───┐ ░ ├───┤ ░       ░ ├───┤ ░  ║ ┌─────┐┌───┐»
q_3: ────────────────░─┤ Z ├─░─┤ X ├─░─┤ H ├─░───────░─┤ H ├─░──╫─┤ Sdg ├┤ H ├»
     ┌───┐           ░ ├───┤ ░ └─┬─┘ ░ ├───┤ ░       ░ ├───┤ ░  ║ ├─────┤├───┤»
q_4: ┤ H ├───────────░─┤ Z ├─░───■───░─┤ X ├─░───────░─┤ H ├─░──╫─┤ Sdg ├┤ H ├»
     └───┘           ░ └───┘ ░       ░ └───┘ ░       ░ └───┘ ░  ║ └─────┘└───┘»
c: 4/═══════════════════════════════════════════════════════════╩═════════════»
                                        

In [341]:
def get_unitary(circuit):
    simulator = Aer.get_backend('unitary_simulator')
    t_circuit = transpile(circuit, simulator)
    qobj = assemble(t_circuit)
    result = simulator.run(qobj)
    return result.get_unitary(circuit, decimals=3)

# Get unitary matrices for each circuit
unitaries = [Clifford_Permute(circuit,qk.quantum_info.Pauli("XYIZX")) for circuit in circuit_ensemble]

# Check if all unitaries are equal
are_equal = all(np.allclose(unitaries[0], unitary) for unitary in unitaries[1:])

if are_equal:
    print("The unitaries of all circuits are equal.")
else:
    print("The unitaries of the circuits are not equal.")

The unitaries of the circuits are not equal.


In [342]:
circuit = generateCliffordCircuit(2, 2, singleGateSet, doubleGateSet)
meme = transpileListToQiskitCircuit(circuit)
print(meme)
from qiskit.transpiler.passes import RemoveBarriers
rb = RemoveBarriers()
meme2 = rb(meme)
print(meme2)

     ┌───┐ ░      
q_0: ┤ S ├─░───■──
     ├───┤ ░ ┌─┴─┐
q_1: ┤ X ├─░─┤ X ├
     └───┘ ░ └───┘
     ┌───┐     
q_0: ┤ S ├──■──
     ├───┤┌─┴─┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘


In [343]:
circuit = generateCliffordCircuit(2, 2, singleGateSet, doubleGateSet)
meme = transpileListToQiskitCircuit(circuit)
print(meme)
p_str = "YX"
print(Clifford_Permute(meme, p_str).to_label())
v = est_Lambda(p_str,[meme],1000)
print(v)

     ┌───┐ ░ ┌───┐
q_0: ┤ H ├─░─┤ X ├
     ├───┤ ░ └─┬─┘
q_1: ┤ I ├─░───■──
     └───┘ ░      
-YZ
-1.0


In [344]:
# circuit = generateCliffordCircuit(2, 2, singleGateSet, doubleGateSet)
# meme = transpileListToQiskitCircuit(circuit)
layers = [x for x in split_circuit_by_barrier(meme)]
print(layers[0])
print(Clifford_Permute(layers[0], "XY"))
v = est_Lambda(p_str,[layers[0]],1000)
print(v)

     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ I ├
     └───┘
YZ
-1.0


In [345]:
a = qk.QuantumCircuit(1)
a.h(0)
print(Clifford_Permute(a,"Y"))

a = qk.QuantumCircuit(1)
a.h(0)
print(Clifford_Permute(a,"X"))

-Y
Z


In [346]:
a = qk.QuantumCircuit(2)
a.x(0)
a.h(1)
a.draw()
p = qk.quantum_info.Pauli("YX")
p.evolve(a, qargs =[0,1], frame="s")

Pauli('-YX')

In [347]:
singleGateSet

['X', 'H', 'Z', 'I', 'S']

In [348]:
dood = {}
for inp in ["X","Y","Z"]:
    for g in singleGateSet:
        a = qk.QuantumCircuit(1)
        if g == "X":
            a.x(0)
        elif g == "Y":
            a.y(0)
        elif g == "Z":
            a.z(0)
        elif g == "H":
            a.h(0)
        elif g == "S":
            a.s(0)
        elif g == "I":
            a.id(0)
        key = ("Input Puali: " + str(inp),"Gate: "+str(g),"Output Pauli: " + str(Clifford_Permute(a,inp)))
        dood[key] = est_Lambda(inp,[a],1000)
        print(key)
        print(dood[key])

print(len(dood))

('Input Puali: X', 'Gate: X', 'Output Pauli: X')
1.0
('Input Puali: X', 'Gate: H', 'Output Pauli: Z')
1.0
('Input Puali: X', 'Gate: Z', 'Output Pauli: -X')
-1.0
('Input Puali: X', 'Gate: I', 'Output Pauli: X')
1.0
('Input Puali: X', 'Gate: S', 'Output Pauli: Y')
1.0
('Input Puali: Y', 'Gate: X', 'Output Pauli: -Y')
-1.0
('Input Puali: Y', 'Gate: H', 'Output Pauli: -Y')
-1.0
('Input Puali: Y', 'Gate: Z', 'Output Pauli: -Y')
-1.0
('Input Puali: Y', 'Gate: I', 'Output Pauli: Y')
1.0
('Input Puali: Y', 'Gate: S', 'Output Pauli: -X')
-1.0
('Input Puali: Z', 'Gate: X', 'Output Pauli: -Z')
-1.0
('Input Puali: Z', 'Gate: H', 'Output Pauli: X')
1.0
('Input Puali: Z', 'Gate: Z', 'Output Pauli: Z')
1.0
('Input Puali: Z', 'Gate: I', 'Output Pauli: Z')
1.0
('Input Puali: Z', 'Gate: S', 'Output Pauli: Z')
1.0
15


In [349]:
import itertools

combinations = list(itertools.product(["X","Y","Z","I"], repeat=2))
combinations = [''.join(tuple_of_strings) for tuple_of_strings in combinations]
combinations.remove("II")
# combinations

In [350]:
dood = {}
for inp in combinations:
    for i in [0,1]:
        a = qk.QuantumCircuit(2)
        if i == 0:  
            a.cx(0,1)
            g = "CNOT(0,1)"
        elif i == 1:
            a.cx(1,0)
            g = "CNOT(1,0)"
        key = ("Input Puali: " + str(inp),"Gate: "+str(g),"Output Pauli: " + str(Clifford_Permute(a,inp)))
        dood[key] = est_Lambda(inp,[a],1000)
        print(key)
        print(dood[key])
print(len(dood))

('Input Puali: XX', 'Gate: CNOT(0,1)', 'Output Pauli: IX')
1.0


('Input Puali: XX', 'Gate: CNOT(1,0)', 'Output Pauli: XI')
1.0
('Input Puali: XY', 'Gate: CNOT(0,1)', 'Output Pauli: ZY')
1.0
('Input Puali: XY', 'Gate: CNOT(1,0)', 'Output Pauli: YI')
1.0
('Input Puali: XZ', 'Gate: CNOT(0,1)', 'Output Pauli: -YY')
-1.0
('Input Puali: XZ', 'Gate: CNOT(1,0)', 'Output Pauli: ZX')
1.0
('Input Puali: XI', 'Gate: CNOT(0,1)', 'Output Pauli: XX')
1.0
('Input Puali: XI', 'Gate: CNOT(1,0)', 'Output Pauli: IX')
1.0
('Input Puali: YX', 'Gate: CNOT(0,1)', 'Output Pauli: IY')
1.0
('Input Puali: YX', 'Gate: CNOT(1,0)', 'Output Pauli: YZ')
1.0
('Input Puali: YY', 'Gate: CNOT(0,1)', 'Output Pauli: -ZX')
-1.0
('Input Puali: YY', 'Gate: CNOT(1,0)', 'Output Pauli: -XZ')
-1.0
('Input Puali: YZ', 'Gate: CNOT(0,1)', 'Output Pauli: YX')
1.0
('Input Puali: YZ', 'Gate: CNOT(1,0)', 'Output Pauli: IY')
1.0
('Input Puali: YI', 'Gate: CNOT(0,1)', 'Output Pauli: XY')
1.0
('Input Puali: YI', 'Gate: CNOT(1,0)', 'Output Pauli: ZY')
1.0
('Input Puali: ZX', 'Gate: CNOT(0,1)', 'Output Pa

In [374]:
# Necessary functions
def makeNoisyGates(qiskitCir, whichQubits, px=0, py=0, pz=0):
    singleOrDouble = len(whichQubits)
    if singleOrDouble == 1:
        gateString = np.random.choice(['I', 'X', 'Y', 'Z'], p=[1-px-py-pz, px, py, pz])
        if gateString == 'I':
            qiskitCir.id(whichQubits[0])
        elif gateString == 'X':
            qiskitCir.x(whichQubits[0])
        elif gateString == 'Y':
            qiskitCir.y(whichQubits[0])
        elif gateString == 'Z':
            qiskitCir.z(whichQubits[0])
    else:
        gateString1 = np.random.choice(['I', 'X', 'Y', 'Z'], p=[1-px-py-pz, px, py, pz])
        if gateString1 == 'I':
            qiskitCir.id(whichQubits[0])
        elif gateString1 == 'X':
            qiskitCir.x(whichQubits[0])
        elif gateString1 == 'Y':
            qiskitCir.y(whichQubits[0])
        elif gateString1 == 'Z':
            qiskitCir.z(whichQubits[0])
        gateString2 = np.random.choice(['I', 'X', 'Y', 'Z'], p=[1-px-py-pz, px, py, pz])
        if gateString2 == 'I':
            qiskitCir.id(whichQubits[1])
        elif gateString2 == 'X':
            qiskitCir.x(whichQubits[1])
        elif gateString2 == 'Y':
            qiskitCir.y(whichQubits[1])
        elif gateString2 == 'Z':
            qiskitCir.z(whichQubits[1])
    return qiskitCir

def transpileListToQiskitCircuit(cir, noise=False, px=0, py=0, pz=0):
    depth = len(cir)
    width = len(cir[0])
    qiskitCir = QuantumCircuit(width)
    for d in range(depth):
        if d % 2 == 0:
            for w in range(width):
                singleGate = cir[d][w]
                stringToQiskitSingleGate(singleGate, qiskitCir, w)
            if noise:
                qiskitCir.barrier()
                for w in range(width):
                    makeNoisyGates(qiskitCir, [w], px, py, pz)
            if d != width - 1:
                qiskitCir.barrier()
        else:
            c = cir[d].index('CNOT_C')
            t = cir[d].index('CNOT_T')
            qiskitCir.cx(c, t)
            if noise:
                qiskitCir.barrier()
                makeNoisyGates(qiskitCir, [c, t], px, py, pz)
            if d != width - 1:
                qiskitCir.barrier()
    return qiskitCir

def stringToQiskitSingleGate(gateString, qiskitCir, whichQubit):
    if gateString == 'I':
        qiskitCir.id(whichQubit)
    elif gateString == 'X':
        qiskitCir.x(whichQubit)
    elif gateString == 'Y':
        qiskitCir.y(whichQubit)
    elif gateString == 'Z':
        qiskitCir.z(whichQubit)
    elif gateString == 'H':
        qiskitCir.h(whichQubit)
    elif gateString == 'S':
        qiskitCir.s(whichQubit)


width = 4  # num_qubits
depth = 3  # number of layers in the circuit (easy + hard)
singleGateSet = ['X', 'H', 'Z', 'I', 'S']
doubleGateSet = ['CNOT_C', 'CNOT_T']
twirlingGateSet = ['X', 'Y' 'Z', 'I']

shots = 100000
# Define a Pauli error channel
px = 0.05  # adjust the probability as needed
py = 0.10  # adjust the probability as needed
pz = 0.20  # adjust the probability as needed

# Generate Clifford circuit and transpile to ideal qiskit circuit
circuit = generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet)
qiskitCircuit =transpileListToQiskitCircuit(circuit)
print(qiskitCircuit)
qiskitCircuitTwirl = G_twirling(qiskitCircuit)
print(qiskitCircuitTwirl)
# Generate Clifford circuit and transpile to noisy qiskit circuit
qiskitCircuitNoisy = transpileListToQiskitCircuit(circuit, noise=True, px=px, py=py, pz=pz)
print(qiskitCircuitNoisy)
# Twirling function
qiskitCircuitTwirlNoisy = G_twirling_noisy(qiskitCircuitNoisy)
print(qiskitCircuitTwirlNoisy)

     ┌───┐ ░       ░ ┌───┐ ░ 
q_0: ┤ I ├─░───────░─┤ X ├─░─
     ├───┤ ░ ┌───┐ ░ ├───┤ ░ 
q_1: ┤ S ├─░─┤ X ├─░─┤ Z ├─░─
     ├───┤ ░ └─┬─┘ ░ ├───┤ ░ 
q_2: ┤ X ├─░───■───░─┤ Z ├─░─
     ├───┤ ░       ░ ├───┤ ░ 
q_3: ┤ I ├─░───────░─┤ Z ├─░─
     └───┘ ░       ░ └───┘ ░ 
     ┌──────────────┐┌───┐┌──────────────┐ ░ ┌──────────────┐     »
q_0: ┤0             ├┤ I ├┤0             ├─░─┤0             ├─────»
     │              │├───┤│              │ ░ │              │┌───┐»
q_1: ┤1             ├┤ S ├┤1             ├─░─┤1             ├┤ X ├»
     │  Pauli(IYYI) │├───┤│  Pauli(IYXI) │ ░ │  Pauli(ZXXZ) │└─┬─┘»
q_2: ┤2             ├┤ X ├┤2             ├─░─┤2             ├──■──»
     │              │├───┤│              │ ░ │              │     »
q_3: ┤3             ├┤ I ├┤3             ├─░─┤3             ├─────»
     └──────────────┘└───┘└──────────────┘ ░ └──────────────┘     »
«     ┌──────────────┐ ░ ┌──────────────┐┌───┐┌──────────────┐ ░ »
«q_0: ┤0             ├─░─┤0             ├┤ X ├┤0   

In [373]:
def G_twirling_noisy(circuit):
    '''This function performs G-twirling on a circuit. It randomly chooses a Pauli from {X,Y,Z,I} and prepends it before a layer, then appends P'=CPC^T after the layer.}
    Inputs: circuit - A Qiskit Circuit object.
    Outputs: new_circuit - A Qiskit Circuit object.'''
    num_qubits = circuit.num_qubits
    new_circuit = qk.QuantumCircuit(num_qubits)
    subcirc_list = list(split_circuit_by_barrier(circuit))
    for index, subcircuit in enumerate(split_circuit_by_barrier(circuit)):
        # if len(subcircuit.data) == 1:
        #     operated_qubits = [subcircuit.find_bit(q).index for q in subcircuit.data[0].qubits]
        # else:
        #     operated_qubits = list(range(num_qubits))
        if index % 2 == 0: #if in odd layer
            pauli_str = ''.join(random.choice(["X","Y","Z","I"]) for _ in range(num_qubits))
            pauli = qk.quantum_info.Pauli(pauli_str[::-1])
            new_circuit = new_circuit.compose(pauli,qubits=range(num_qubits))
            new_circuit = new_circuit.compose(subcircuit,qubits=range(num_qubits))
        if index % 2 != 0: #if in even layer
            new_circuit = new_circuit.compose(subcircuit,qubits=range(num_qubits))
            new_circuit = new_circuit.compose(Clifford_Permute(subcirc_list[index-1],pauli).to_instruction(),qubits = range(num_qubits))
            if index != len(list(split_circuit_by_barrier(circuit)))-1:
                new_circuit.barrier()
    return new_circuit

In [ ]:
noisy_circuit_ensemble = [G_twirling_noisy(qiskitCircuit) for _ in range(10)]